In [2]:
#if working on lx-moissard
Cluster = 'Occ/'
run_name = '20_08_18_new_big_one_0'
filepath = '/data/Lathys/Visualisation/' + Cluster + run_name + '/ncfiles/p3_files/'

numbers = ["4469", "4530", "4529", "4409", "4410", "4470"]

p3_files = [filepath + "p3_" + number + "_18_08_20_t00230.nc" for number in numbers]

In [5]:
import netCDF4 as nc
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rcParams

rcParams["figure.figsize"] = [20, 15]

In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/clement/.local/lib/python3.6/site-packages/matplotlib/mpl-d

In [17]:
a = list(np.array([1,2,3]))

In [18]:
a = a+a
print(a)

[1, 2, 3, 1, 2, 3]


In [3]:
posx = []
posy = []
posz = []

velx = []
vely = []
velz = []

for p3_file in p3_files:

    p3_data = nc.Dataset(p3_file,'r')
    
    posx = posx + list(np.array(p3_data.variables['particule_x']))
    posy = posy + list(np.array(p3_data.variables['particule_y']))
    posz = posz + list(np.array(p3_data.variables['particule_z']))

    velx = velx + list(np.array(p3_data.variables['particule_vx']))
    vely = vely + list(np.array(p3_data.variables['particule_vy']))
    velz = velz + list(np.array(p3_data.variables['particule_vz']))


FileNotFoundError: [Errno 2] No such file or directory: b'/data/Lathys/Visualisation/Occ/20_08_18_new_big_one_0/ncfiles/p3_files/p3_0000_18_08_20_t00230.nc'

In [ ]:
plt.scatter(posx, posy, s = 0.01)

In [ ]:
plt.scatter(posx, posz, s = 0.01)